In [ ]:
import io

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy
import scipy.stats as st
from theano import tensor as tt

# Model building and expansion for coho salmon

To motivate thinking through problems in a Bayesian context, we're going to work through the 2003 work of Nick Barrowman and others *THE VARIABILITY AMONG POPULATIONS OF COHO SALMON IN THE MAXIMUM REPRODUCTIVE RATE AND DEPENSATION*, pubished in [Ecology in 2003](https://esajournals.onlinelibrary.wiley.com/doi/full/10.1890/1051-0761%282003%29013%5B0784%3ATVAPOC%5D2.0.CO%3B2). This is a spectacular piece of work, from a time when non-linear mixed-effects and Bayesian models were few and far between. [Nick](https://nick-barrowman.carrd.co/) went on to a successful careear in clinical research, using his modelling skills to great effect in Toronto (but a loss for fisheries ecology).




In [ ]:
# coho data from Barrowman et al. (2003)
coho_data = pd.read_csv('./data/bman_coho.csv', index_col=0)

In [ ]:
coho_data.head()

Exercises:
----------

1. How many observations do we have total? 
2. What is the maximum and minimum number of observations?

In [ ]:
sample_rivers = coho_data.river.unique()

fig, axes = plt.subplots(5, 3, figsize=(9, 9), constrained_layout=True)
axes = axes.ravel()

for i,r in enumerate(sample_rivers):
    y = coho_data.R[coho_data.river==r]
    x = coho_data.S[coho_data.river==r]
    axes[i].scatter(x, y, c='black')
    
    axes[i].set_title(r)
    axes[i].set_ylim(0,max(y)*1.05)
    axes[i].set_xlim(0,max(x)*1.05)

axes[i+1].set_axis_off()

fig.text(0.5, -0.04, 'S(number of spawning females per kilometer of river)', ha='center', fontsize=15)
fig.text(-0.04, 0.5, 'R(number of female smolts per kilometer of river)', va='center', rotation='vertical', fontsize=15);

After plotting, we see that generally there are more recruits (R) where there are more spawners (S), but its also kind of all over the place at times.

1. What are some questions we might want to answer with this data set?
2. Suppose we could predict the number of smolts per spawner in each river. What could we do with that information?
3. What are some other pieces of data we might be interested in, if we could go back and design the data collection process?

## Beverton-Holt model

First we will fit a traditional Beverton-Holt stock-recruitment model. If you don't know what these chaps accomplished, they invented modern fisheries science, using cardboard models and an adding machine confiscated from the Nazi's:

![ray_sid](images/ray_sid.jpg)

Incidentally, Sidney Holt was a remarkable guy, who only recently [died at 93](https://www.theguardian.com/environment/2020/jan/08/sidney-holt-obituary), went on to literally save the whales, and held on to that adding machine:

![holt](images/holt.jpg)


Beverton and Holt's standard stock recruitment model can be given by:

$$
R_{i,t}=\frac{\alpha_{i}S_{i,t}}{1+S_{i,t}/K_{i}}
$$

where the number of recruits (R) is proportional to the recruitment per spawner ($\alpha$), the number of spawners (S), and the carrying capacity of the environment (K). Noting that $R_{max,i}\equiv \alpha_{i}/K_{i}$ Barrowman et al. re-write the model by dividing by $\alpha$:

$$
R_{i,t}=\frac{S_{i,t}}{1/\alpha_{i}+S_{i,t}/R_{max,i}}
$$

Here is how to write that model in PyMC3.

In [ ]:
coho_data.head()

As we need to fit an individual model for each creek or river as a baseline comparison, we need a creek index for each observation

In [ ]:
# Helper function
def indexall(L):
    poo = []
    for p in L:
        if not p in poo:
            poo.append(p)
    Ix = np.array([poo.index(p) for p in L])
    return poo,Ix

In [ ]:
Creek,Ic = indexall(coho_data.river.values)
ncreek = len(Creek)

And let's grab the observations while we're at it

In [ ]:
Robs = coho_data.R.values
Sobs = coho_data.S.values

In [ ]:
with pm.Model() as BH_unpooled:
    α = pm.Normal('alpha', 0, 10, shape=ncreek)
    Rmax = pm.Normal('Rmax', 0, 10, shape=ncreek)
    K = pm.Deterministic('K', α/Rmax)

    # Basic Beverton-Holt
    μ = Sobs/(1/α[Ic]+Sobs/Rmax[Ic])
    
    # Common variance
    σ = pm.Exponential('sigma',1)

    # Likelihood
    recruits = pm.Normal('recruits', μ, σ, observed=np.log(Robs))

In [ ]:
from pymc3 import sample_prior_predictive as SPP

with BH_unpooled:
    
    prior_checks = SPP(samples=1000)

In [ ]:
plt.hist((prior_checks["recruits"]))
#plt.hist(coho_data.R.values, alpha=0.2,  color='black')
plt.xlabel("Observed female smolts per river km")
plt.ylabel("Prior predicted female smolts per river km");

Whoops - what happened here? Our counts can be any positive number - we don't need a model for that! Let's think about what reasonable priors might be. 

For Rmax we're talking about the asymptotic level of median recruitment, so it needs to be some postive number, in and around the maximum observed in the data. Remebering the Normal model is for log recruits, we'll take the expected value to be positive, somewhere around 1000 or so. N(7,2) gives us 90% of expected values between about 40 and 30,000:

In [ ]:
import scipy as sp
np.exp(sp.stats.norm.ppf((0.05, .95), 7, 2))

The $\alpha$ parameter is the slope at the origin of the stock-recruitment relationship, which is positive. So let's pick something around 5 and see what happens. N(5,3) gives us 90% of expected values between about 28 and 770:

In [ ]:
np.exp(sp.stats.norm.ppf((0.05, .95),5,1))

In [ ]:
with pm.Model() as BH_unpooled:
    α = pm.Normal('alpha', 5, 1, shape=ncreek)
    Rmax = pm.Normal('Rmax', 7, 2, shape=ncreek)
    K = pm.Deterministic('K', α/Rmax)

    # Basic Beverton-Holt
    μ = Sobs/(1/α[Ic]+Sobs/Rmax[Ic])
    
    # Common variance
    σ = pm.Exponential('sigma',1)

    # Likelihood
    recruits = pm.Normal('recruits', μ, σ, observed=np.log(Robs))

In [ ]:
from pymc3 import sample_prior_predictive as SPP

with BH_unpooled:
    
    prior_checks = SPP(samples=1000)

In [ ]:
max(np.exp(prior_checks["recruits"].flatten()))

In [ ]:
plt.hist(prior_checks["recruits"].flatten())
#plt.hist(coho_data.R.values, alpha=0.2,  color='black')
plt.xlabel("Prior predicted log(female smolts) per river km");

These still give some wild values ($e^{15}=3,269,017$) that we may want to address later but for the most part the log(Recruits) are bewteen 1 and 20,000 or so. Next we can sample from our model and see how we do:

In [ ]:
with BH_unpooled:
    BH_unpooled_trace = pm.sample(1000, progressbar=True)

In [ ]:
unpooled_alpha = pd.Series(BH_unpooled_trace['alpha'].mean(axis=0), index=Creek)
unpooled_Rmax = pd.Series(BH_unpooled_trace['Rmax'].mean(axis=0), index=Creek)
unpooled_K = pd.Series(BH_unpooled_trace['K'].mean(axis=0), index=Creek)

In [ ]:
fig, axes = plt.subplots(5, 3, figsize=(9, 9), constrained_layout=True)
axes = axes.ravel()

for i,c in enumerate(Creek):
    y = Robs[Ic==i]
    x = Sobs[Ic==i]
    axes[i].scatter(x, y, c='black')
    axes[i].set_title(c)
    axes[i].set_ylim(0,max(y)*1.05)
    axes[i].set_xlim(0,max(x)*1.05)
    
    # Add posterior estimates
    x_ = np.linspace(0,max(x),100)
    y_ = np.exp(x_/(1/unpooled_alpha[c]+x_/unpooled_Rmax[c]))
    axes[i].plot(x_,y_,'blue')

axes[i+1].set_axis_off()

fig.text(0.5, -0.04, 'S(number of spawning females per kilometer of river)', ha='center', fontsize=15)
fig.text(-0.04, 0.5, 'R(number of female smolts per kilometer of river)', va='center', rotation='vertical', fontsize=15);

For a first-cut, with individual fits these look not too bad. Let's see what we get if we build some hierarhcy into them, using a partial-pooled model; is there an advantage?

In [ ]:
with pm.Model() as BH_partialpool:
    # Alpha model
    α = pm.Normal('alpha', 5, 1)
    σα = pm.Exponential('sigma_alpha',1)
    α_i = pm.Normal('alpha_i', α, σα, shape=ncreek)
    
    # Rmax model
    Rmax = pm.Normal('Rmax', 7, 2)
    σRmax = pm.Exponential('sigma_Rmax',1)
    Rmax_i = pm.Normal('Rmax_i', Rmax, σRmax, shape=ncreek)
    
    # Calculate K
    K = pm.Deterministic('K', α/Rmax)
    K_i = pm.Deterministic('K_i', α_i/Rmax_i)

    # Basic Beverton-Holt
    μ = Sobs/(1/α_i[Ic]+Sobs/Rmax_i[Ic])
    
    # Common variance
    σ = pm.Exponential('sigma',1)

    # Likelihood
    recruits = pm.Normal('recruits', μ, σ, observed=np.log(Robs))

In [ ]:
with BH_partialpool:
    BH_partialpool_trace = pm.sample(1000, progressbar=True)

In [ ]:
Ppooled_alpha = pd.Series(BH_partialpool_trace['alpha_i'].mean(axis=0), index=Creek)
Ppooled_Rmax = pd.Series(BH_partialpool_trace['Rmax_i'].mean(axis=0), index=Creek)
Ppooled_K = pd.Series(BH_partialpool_trace['K_i'].mean(axis=0), index=Creek)

In [ ]:
fig, axes = plt.subplots(5, 3, figsize=(9, 9), constrained_layout=True)
axes = axes.ravel()

for i,c in enumerate(Creek):
    y = Robs[Ic==i]
    x = Sobs[Ic==i]
    axes[i].scatter(x, y, c='black')
    axes[i].set_title(c)
    axes[i].set_ylim(0,max(y)*1.05)
    axes[i].set_xlim(0,max(x)*1.05)
    
    # Add posterior estimates
    x_ = np.linspace(0,max(x),100)
    y_ = np.exp(x_/(1/unpooled_alpha[c]+x_/unpooled_Rmax[c]))
    axes[i].plot(x_,y_,'blue')
    y_2 = np.exp(x_/(1/Ppooled_alpha[c]+x_/Ppooled_Rmax[c]))
    axes[i].plot(x_,y_2,'blue',ls=":")

axes[i+1].set_axis_off()

fig.text(0.5, -0.04, 'S(number of spawning females per kilometer of river)', ha='center', fontsize=15)
fig.text(-0.04, 0.5, 'R(number of female smolts per kilometer of river)', va='center', rotation='vertical', fontsize=15);

A nice feature in some packages is to look at a Directed Acyclic Graph (DAG) of our model, to see how our hierarhical model structure can be represented visually:

In [ ]:
pm.model_to_graphviz(BH_partialpool)

## Model checking

Of all the model checks to do, by far the most important is the one we have already done - by plotting fitted curves against our data we have taken a big step to feeling confident that we have accomplished something.

The next most important check is for convergence - the holy grail of Bayesian statistics in that we can never know if a sampler has converged for certain. We can only be really really sure. First a look at the posterior traces:

In [ ]:
pm.traceplot(BH_partialpool_trace, var_names=['alpha','alpha_i','sigma_alpha']);

In [ ]:
pm.traceplot(BH_partialpool_trace, var_names=['Rmax','Rmax_i','sigma_Rmax']);

In [ ]:
pm.traceplot(BH_partialpool_trace, var_names=['sigma']);

While not perfect, these aren't the worst either; it looks like one or two of the four chains had problems mixing during the end. It could be a one-off, or indicidative of the need to re-parameterize. We'll call it ok for now.

Another model check is to formally compare estimates from the four independent MCMC chains, comparing the distribution of the variances to see if they are stable and smiliar. If so, the $\hat{R}$ ratio should be at or near 1:

In [ ]:
pm.forestplot(BH_partialpool_trace,r_hat=True);

From these posterior checks, we can be reasonably comfortable that things are well-enough behaved to proceed. Note the divergences above suggest we will might want to tweak the model at some point, either re-parameterizing or changing the [tuning algorithms used](https://docs.pymc.io/notebooks/Diagnosing_biased_Inference_with_Divergences.html) (which is possibly above the heads of many users), although there are quite few of these:

In [ ]:
len(BH_partialpool_trace['diverging'].nonzero())/len(BH_partialpool_trace['diverging'])

But, the energy plot from the NUTS sampler has heaver tails for one of the two energy distrubitons, indiciating we have sampler issues that should eventually get sorted out.

In [ ]:
pm.energyplot(BH_partialpool_trace);


## Posterior predictive checks

Now that we have sampled parameters, we can use `PyMC3` to sample observations from the posterior distribution. This can be compared to prediction in classical statistics or machine learning.

In [ ]:
with BH_partialpool:
    ppc = pm.sample_posterior_predictive(BH_partialpool_trace)

The great thing about a Bayesian model is that each observation itself gets a posterior predictive distribution, meaning we can check each observation and how well they're being predicted:

In [ ]:
# Setup multipanel figure
nrows, ncols = 5, 5
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12,8))
ax_ = axes.flatten()

# Transform ppc
PPC = ppc['recruits'].T
# Index random observations
indx = np.random.randint(0,len(Robs),size=nrows*ncols)

# Iterate over random samples from ppc
for i in range(nrows*ncols):
    ax1 = ax_[i]
    ax1.hist(PPC[indx[i]], bins=19, alpha=0.5)
    ax1.axvline(np.log(Robs[indx[i]]),c='red')

fig.tight_layout()
fig.suptitle('Posterior predictive distributions', fontsize=14)
fig.subplots_adjust(top=0.95);

The red vertical line is the observed data and the blue distribution is the posterior predictive distribution for that observation - these look great. In principle we'd expect 5% of observations to lie outside the 95% uncertainty interval for each PPD:

In [ ]:
tmp = 0
for i in range(len(Robs)):
    if np.log(Robs)[i]<np.quantile(PPC[i],.025) or np.log(Robs)[i]>np.quantile(PPC[i],.975):
        tmp += 1
    else:
        pass

In [ ]:
tmp/len(Robs)

We can also look at the average number of log-recruits in the data and see how well our average PPD's do in terms of their distribution of means:

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))
ax.hist([n.mean() for n in ppc['recruits']], bins=19, alpha=0.5)
ax.axvline(np.mean(np.log(Robs)))
ax.set(title='Posterior predictive of the mean', xlabel='mean(x)', ylabel='Frequency');

We can also look at the accuracy the model predicts at various levels of spawners compared to actual accuracy, and perhaps use that to improve the model if there are systematic or obvious errors.

In [ ]:
residuals = (np.log(Robs) - ppc['recruits'].mean(axis=0))

fig, ax = plt.subplots()

ax.plot(Sobs, residuals, 'o')
ax.axhline(y=0, linestyle='dashed', linewidth=1)
ax.set_xlabel("Spawners");

This shows us that the deviance between the observed and predicted numbers of log-recruits is higher at lower numbers of spawners. While this likely means the model is chasing these rarer high value datapoints, it isn't necessarily a problem. Most of the observations are less than 50, where the variance is mostly concentrated between [-0.5, 0.5].

## Model comparison

Now that we have checked (one of) our models, a valid question is to ask if buidling the hierarhical model has made much difference - the fits are pretty close. We can do this using typically either the widely-acceptable information criterion (WAIC) or through Pareto-smoothed leave-one-out cross validation (LOO). See [Vehtari et al 2015](https://arxiv.org/abs/1507.04544) for details.

In [ ]:
import arviz as ar

In [ ]:
BH_compare = ar.compare({"Individual fits": BH_unpooled_trace, "Hierarhical": BH_partialpool_trace}, ic='LOO')
BH_compare

The results give us a range of information to use for comparing these two models:

1. The **rank** column ranks the models acording to the lowest loo value.

2. The **loo** column  contains the values of loo (or WAIC).

3. The **p_loo** column is the estimated effective number of parameters. In general, models with more parameters will be more flexible to fit data and at the same time could also lead to overfitting. Thus we can interpret p_loo as a penalization term, intuitively we can also interpret it as measure of how flexible each model is in fitting the data.

4. The **weight** column is useful sometimes when comparing models, if do not want to select the “best” model, and instead we want to perform predictions by averaging along all the models (or at least several models). Ideally we would like to perform a weighted average, giving more weight to the model that seems to explain/predict the data better. There are many approaches to perform this task, one of them is to use Akaike weights based on the values of loo (or WAIC) for each model. These weights can be loosely interpreted as the probability of each model (among the compared models) given the data.

5. The se column records the standard error for the loo (or WAIC) computations. The standard error can be useful to assess the uncertainty of these estimates. Nevertheless, caution need to be taken because the estimation of the standard error assumes normality and hence could be problematic when the sample size is low.

6. In the same way that we can compute the standard error for each value of loo (or WAIC), we can compute the standard error of the differences between two values of loo (or WAIC) in the **dse** column. Notice that both quantities are not necessarily the same, the reason is that the uncertainty about loo is correlated between models. This quantity is always 0 for the top-ranked model.

7. The **warning** column contains value of 1 (or True) indicating that the computation of loo (or WAIC) may not be reliable, this warning is based on an empirical determined cutoff value and need to be interpreted with caution. It is possible to look at each observation and (with care) remove observations that fail the cutofff. For more details you can read [Vehtari et al 2015](https://arxiv.org/abs/1507.04544).

PyMC aslo has a nice port of a plot from Richard MacElreath's [Statistical Rethinking book](http://xcelab.net/rm/statistical-rethinking/), which is excellent.

In [ ]:
pm.plots.compareplot(BH_compare)

This provides a few bits of information:

1. The filled points are in-sample (within our data) deviance.
2. The open points are the loo (or WAIC) values, which include the deviance and some various penalties for out of sample prediction.
3. The line segments are the standard errors of loo (or WAIC) in the **se** column above.
4. The filled triangle is the standard error of the difference in loo (or WAIC) between the models.
5. The line through the triangle is the standard error of the difference, **dse** in the table above.
6. The dashed vertical line is the lowest-ranked loo (or WAIC) value.

The triangle is the money here - the fact that it doesn't overlap with the dashed vertical line suggestes the Hierarhical model has substantailly more support, given the data.

## A new model

In addition to these models, Barrowman et al. (2003) had an additional, more complex model that allowed for dispensation: *a decrease in the breeding population (mature individuals) leads to reduced production and survival of eggs or offspring* [Wikipedia](https://en.wikipedia.org/wiki/Depensation). To allow for this, they added a dispensation term to the model ($\delta$):


$$
R_{i,t}=\frac{\alpha_{i}S_{i,t}^{\delta_{i}}}{1+S_{i,t}^{\delta_{i}}/K_{i}}
$$

Which can be re-written as:


$$
R_{i,t}=\frac{S_{i,t}^{\delta_{i}}}{1/\alpha_{i}+S_{i,t}^{\delta_{i}}/R_{max,i}}
$$

Barrowman didn't implement this type-1 model (they used a type-2 model instead, where the Beverton-Holt model is reduced by $S_{i,t}/(S_{i,t}+d_{i})$). Let's give it a go. 


In [ ]:
# Code your type-1 dispensation model here

In [ ]:
# Run your sampler here

In [ ]:
# Compare the LOO weights here

---
## References

McElreath, R. (2018) Statistical rethinking: A Bayesian course with examples in R and Stan. Chapman and Hall/CRC, 2018.

Vehtari, A., Gelman, A., & Gabry, J. (2017). Practical Bayesian model evaluation using leave-one-out cross-validation and WAIC. Statistics and computing, 27(5), 1413-1432.

Barrowman, N. J., Myers, R. A., Hilborn, R., Kehler, D. G., & Field, C. A. (2003). The variability among populations of coho salmon in the maximum reproductive rate and depensation. Ecological Applications, 13(3), 784-793.